## Project 3:  Sequence Alignment 

##### Due Friday May 13

If two or more people are working on this project enter each person's name here:
* Name:
* Name:
* Name:

Note: only one person needs to submit the project via Canvas

### Imports 

For this project I suggest you use `skbio` to read sequences to align from FASTA files and `numpy` to make the dynamic programming matrix.

In [1]:
import skbio
import numpy as np
import pandas as pd

/Users/hrnmy/anaconda3/lib/python3.4/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


### A Note about "Tags" 

In lecture on Friday (April 29) I described a scheme that would let us combine scores and traceback information in the dynamic programming algorithm into a single matrix of integers by adding 2-bit tags to each matrix entry.

The project description in this notebook is based on this scheme, and the easiest way to complete this project is to fill in all the function definitions as described.

You do not have to use this scheme, however.  If you'd prefer to use the traditional approach, with separate matrices for scores and tracebacks (or maybe an object-oriented approach, where the matrix is a matrix of objects that have score and direction attributes) you are free to do so.  If you want to create your own representations:
* you can skip Part 2 (helper functions for tags)
* you must complete Part 3 (`print_matrix` functions) but you can modify the definition of the function so it is passed the information in your representation, e.g. pass it separate score and traceback matrices
* you are on your own for designing the code for the last two parts (creating the score matrix, following tracebacks to generate an alignment)
* your `dp_matrix` function should return both score and traceback matrices; you will need to modify the unit tests so they work with your representation

**Important:** &nbsp; If you do choose your own representation for scores and tracebacks fill in the following markdown cell to explain your representation and alert me so I give you back the points the autograder will deduct for skipping Part 2 and any modifications to unit tests.

In the end, made separate traceback matrix from dynamic programming matrix! all parts filled in, but dp_matrix returns two matrices. 

### Part 1: &nbsp; Substitution Matrix 

Below is the outline of a class named SubstitutionMatrix.  An object of this class will be a 2D matrix where coordinates are amino acid letters, and the values in the matrix are integers that are based on how similar the corresponding amino acids are.  Positive numbers represent a high similarity, and negative numbers mean low similarity.  

When the constructor is called it should be passed the name of a file that has an amino acid substitution matrix in the format used by NCBI BLAST:
* the file can have any number of comment lines (lines beginning with `#`); these are ignored
* the first line contains a list of amino acid letters
* the remaining lines have rows of the matrix

Each row starts with an amino acid letter, and the rest of the row has matrix entries.  You need to scan these lines and use the location of a number within a line to figure out where to place it in the matrix.

For example, suppose the list of letters on the first line starts like this:
```
   A  R  N  D ...
```
That tells you the first score in each row goes in the `A` column, the next in the `R` column, and so on.  When you read a row that looks like this:
```
R  -2  7  -1 -2 ...
```
you will set the entry for row R, column A to -2; row R, column R to 7, and so on.

This example shows how to make a SubstitutionMatrix object based on the BLOSUM50 matrix, which is in a file with the same name (you can download the file from Canvas):
```
>>> m = SubstitutionMatrix('BLOSUM50')
```

You need to fill in the definition of the `__getitem__` method so it returns a score for a pair of letters.  For example, 
```
>>> m['A','R']
-2

>>> m['R','R']
7
```

You also need to fill in the bodies of the methods named `keys`, which returns a string containing the amino acid letters, and `similarity`, which returns a letter based on how similar two amino acids are.  A call to `m.similarity(X,Y)` returns `:` if X is the same as Y, `.` if the matrix entry is positive, and a space if the matrix entry is negative:
```
>>> m.keys()
'ARNDCQEGHILKMFPSTWYVBZX*'

>>> m.similarity('R','R')
':'

>>> m.similarity('D','E')
'.'

>>> m.similarity('D','W')
' '
```

Two other methods are optional.  Fill in the `__repr__` method so it displays a matrix in readable form, similar to what it looks like in the file, and `description` so it prints the comments that were in the file.

**Note:** &nbsp;  Even though this class is called a "matrix", and it looks and behaves like a 2D array, you can use any Python object to represent the data internally, including a dictionary.

##### Documentation and Code

With file open, if line is first line after comments, set that to keys to use for returning the keys in keys() function and also in making dictionary keys. Matrix is stored in a dictionary of dictionaries for each amino acid. After first line, create an empty dictionary for an amino acid 'a' and use python's zip function in order to go through each amino acid in keys and assign new key for amino acid 'b' and set value as score for 'a, b'.

description also set by returning commented lines.

In [2]:
class SubstitutionMatrix:
    
    def __init__(self, filename):
        
        data = open(filename).readlines()

        self._matrix = {}
        first = True
        self._description = ''
        self._keys = []

        for line in data:
            if not line.startswith('#'):
                line = line.strip('/n/r').split()
                if first:
                    self._keys = line
                    first = False
                else: 
                    a = line[0]
                    self._matrix[a] = {}
                    for i, b in zip(self._keys, line[1:]):
                        self._matrix[a][i] = b
            else:
                self._description += line[1:]

    def __repr__(self):
        # YOUR CODE HERE
        return pd.DataFrame(self._matrix).to_string()
    
    def description(self):
        # YOUR CODE HERE
        print(self._description)
    
    def __getitem__(self, key):
        # YOUR CODE HERE
        ch1, ch2 = key
        return int(self._matrix[ch1][ch2])
    
    def keys(self):
        # YOUR CODE HERE
        #keys = self.
        return ''.join(self._keys)
    
    def similarity(self, ch1, ch2):
        # YOUR CODE HERE
        entry = int(self._matrix[ch1][ch2])
        
        if ch1 == ch2:
            return ':'
        elif entry < 0:
            return ' '
        else:
            return '.'
    

In [3]:
test = SubstitutionMatrix('GONNET')

In [4]:
test

    *   A   C   D   E   F   G   H   I   K   L   M   N   P   Q   R   S   T   V   W   X   Y
*   1  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8  -8
A  -8   2   0   0   0  -2   0  -1  -1   0  -1  -1   0   0   0  -1   1   1   0  -4   0  -2
C  -8   0  12  -3  -3  -1  -2  -1  -1  -3  -2  -1  -2  -3  -2  -2   0   0   0  -1  -3   0
D  -8   0  -3   5   3  -4   0   0  -4   0  -4  -3   2  -1   1   0   0   0  -3  -5  -1  -3
E  -8   0  -3   3   4  -4  -1   0  -3   1  -3  -2   1   0   2   0   0   0  -2  -4  -1  -3
F  -8  -2  -1  -4  -4   7  -5   0   1  -3   2   2  -3  -4  -3  -3  -3  -2   0   4  -2   5
G  -8   0  -2   0  -1  -5   7  -1  -4  -1  -4  -4   0  -2  -1  -1   0  -1  -3  -4  -1  -4
H  -8  -1  -1   0   0   0  -1   6  -2   1  -2  -1   1  -1   1   1   0   0  -2  -1  -1   2
I  -8  -1  -1  -4  -3   1  -4  -2   4  -2   3   2  -3  -3  -2  -2  -2  -1   3  -2  -1  -1
K  -8   0  -3   0   1  -3  -1   1  -2   3  -2  -1   1  -1   2   3   0   0  -2  -4  -1  -2
L  -8  -1 

##### Autograder Tests 

The autograder will run these tests in the order shown.  Feel free to use these tests yourself, but do not alter the cells or reorder them.

In [4]:
m = SubstitutionMatrix('GONNET')
assert m.keys() == 'CSTPAGNDEQHRKMILVFYWX*'

In [5]:
assert m['C','C'] == 12

In [6]:
assert m['C','Q'] == -2

In [7]:
assert sum([sum([m[ch1,ch2] for ch1 in m.keys()]) for ch2 in m.keys()]) == -659

In [8]:
assert m.similarity('A','A') + m.similarity('A','T') + m.similarity('A','W') == ':. '

### Part 2: &nbsp; Helper Functions

Before you write the functions that create the dynamic programming matrix and follow the traceback to print the alignment you need to define some "helper functions".

Here is a trick that will let you manage all the data in a single matrix instead of having separate matrices for scores and tracebacks.  Create a matrix of integers for scores, but before you put a score in the matrix add a 2-bit tag for the traceback.  If the tag is in the low order 2 bits you don't need to remove it when comparing matrix entries.

Execute this code cell to define the constants used for tag values:

In [9]:
LEFT = 2
UP = 1
DIAG = LEFT + UP

Fill in the bodies of the functions below.
* pass `mark` a score and a tag, and it will return the tagged integer to add to the matrix
* call `value` or `tag` to get the score or tag from a tagged matrix entry
* `unpack` can be used in cases where both values are needed, to avoid having to make two function calls to get the value and tag from a tagged matrix entry

Example:  the binary representation of 7 is `111`.  To put it in the matrix with an UP tag:
```
>>> mark(7, UP)
29
```

If we write the return value in binary we'd see `11101`: the number 7 has been shifted over to make room for the tag, and the tag is in the low-order 2 bits.

**Note:** &nbsp; No documentation is required for these functions.

In [10]:
def mark(value, direction):
    '''Combine value and direction into an integer to store in the dynamic programming matrix'''
    # YOUR CODE HERE
    x = (value << 2) | direction
    return x

def value(cell):
    '''Return the value field of a matrix cell'''
    # YOUR CODE HERE
    val = cell >> 2
    return val
    
def tag(cell):
    '''Return the tag field of a matrix cell'''
    # YOUR CODE HERE
    direction = cell & 3
    
    return direction
    
def unpack(cell):
    '''Return the value and tag fields from a matrix cell'''
    # YOUR CODE HERE
    return value(cell), tag(cell)

In [11]:
mark(7, UP)

29

In [12]:
DIAG

3

In [13]:
tag(0b11101)

1

In [14]:
unpack(0b1000011)

(16, 3)

In [15]:
0b11

3

In [16]:
value(mark(-80, LEFT))

-80

In [17]:
assert mark(7, UP) == 0b11101
assert mark(8, LEFT) == 0b100010 
assert mark(16, DIAG) == 0b1000011

In [18]:
for score in [7, 8, 9]:
    for direction in [UP, LEFT, DIAG]:
        x = mark(score, direction)
        assert value(x) == score
        assert tag(x) == direction
        assert unpack(x) == (score, direction)

### Part 3: &nbsp; Print Matrix

Another helper function is one that will display the dynamic programming matrix, showing each cell value along with its direction tag (this project will also help you make sure your marking and unpacking functions are working).  

Fill in the body of the `print_matrix` function below so it displays a matrix.  The only required argument is a matrix of integers.  Two optional arguments are strings to use for row and column labels.

The easiest choice is to just print the rows, using a format similar to that shown in the lecture notes, but you can use Matplotlib or any other method.

Example output (using the format shown in lecture notes, and assuming `m` is a dynamic programming matrix filled with tagged integers):
```
>>> print_matrix(m, 'MLS', 'MIW')

    󠀠       󠀠  M    󠀠  I    󠀠  W 
   •   0  ↤  -8  ↤ -16  ↤ -24
M  ↥  -8  ⇖   7  ↤  -1  ↤  -9
L  ↥ -16  ↥  -1  ⇖   9  ↤   1
S  ↥ -24  ↥  -9  ↥   1  ⇖   5
```

**Note:** &nbsp; No documentation is required for these functions.

In [271]:
# Uncomment one of the definitions below to define the symbols to print
# for traceback directions (or define your own symbols)

arrow = { UP: '↥', LEFT: '↤', DIAG: '⇖', 0: '•'}
#arrow = { UP: '^', LEFT: '<', DIAG: '\\', 0: ' '}

def print_matrix(m, t, S=None, T=None):
    # YOUR CODE HERE
    
    for i,j in zip(m,t):
        for h,k in zip(i,j):
            val, tag = h, k
            print(arrow[tag], val, '', end = '')
        print('\n')

In [ ]:
m

In [389]:
print_matrix(D, T, 'AGK', 'FJAKL')

• 0 ↤ -8 ↤ -16 ↤ -24 ↤ -32 ↤ -40 ↤ -48 ↤ -56 ↤ -64 ↤ -72 ↤ -80 

↥ -8 ⇖ -2 ⇖ -9 ↤ -17 ↤ -25 ↤ -33 ↤ -41 ↤ -49 ↤ -57 ↤ -65 ↤ -73 

↥ -16 ↥ -10 ⇖ -3 ⇖ -4 ↤ -12 ↤ -20 ↤ -28 ↤ -36 ↤ -44 ↤ -52 ↤ -60 

↥ -24 ↥ -18 ↥ -11 ⇖ -6 ⇖ -7 ↤ -15 ⇖ -5 ↤ -13 ↤ -21 ↤ -29 ↤ -37 

↥ -32 ⇖ -14 ⇖ -18 ⇖ -13 ⇖ -8 ⇖ -9 ↥ -13 ⇖ -7 ⇖ -3 ↤ -11 ↤ -19 

↥ -40 ↥ -22 ⇖ -8 ↤ -16 ↥ -16 ⇖ -9 ⇖ -12 ↥ -15 ⇖ -7 ⇖ 3 ↤ -5 

↥ -48 ↥ -30 ↥ -16 ⇖ -3 ↤ -11 ⇖ -11 ⇖ -12 ⇖ -12 ↥ -15 ↥ -5 ⇖ 2 

↥ -56 ↥ -38 ↥ -24 ↥ -11 ⇖ -6 ⇖ -12 ⇖ -14 ⇖ -15 ⇖ -12 ⇖ -9 ⇖ 1 



##### Autograder Tests 

The autograder will run these three cells.  The first defines a small matrix and adds entries using the helper functions from the previous section, the next two call your `print_matrix` function.  Add additional test cells if you want, but make sure these three cells are here.

In [23]:
seq1 = 'AB'
seq2 = 'CD'
m = np.zeros((len(seq1)+1, len(seq2)+1), dtype=np.int32)
m[0,1] = m[0,2] = mark(0, LEFT)
m[1,0] = m[2,0] = mark(0, UP)
m[1,1] = mark(1, DIAG)
m[1,2] = mark(2, DIAG)
m[2,1] = mark(3, DIAG)
m[2,2] = mark(4, DIAG)

In [24]:
print_matrix(m)

• 0 ↤ 0 ↤ 0 

↥ 0 ⇖ 1 ⇖ 2 

↥ 0 ⇖ 3 ⇖ 4 



In [25]:
print_matrix(m, seq1, seq2)

• 0 ↤ 0 ↤ 0 

↥ 0 ⇖ 1 ⇖ 2 

↥ 0 ⇖ 3 ⇖ 4 



### Part 4: &nbsp; Dynamic Programming Matrix

Fill in the body of the function below so it creates the dynamic programming matrix for the global alignment of two protein sequences.  The arguments passed to the function will be two sequences, `S` and `T`, a substition matrix, and a gap penalty.  Use the recurrence shown in lecture slides fill in the values of the matrix, and return the matrix as the value of the function call.

**Note:** &nbsp; Do not assume `S` and `T` are strings.  The function might be passed Protein objects from skibo or other "string-like" objects. 

**Note:** &nbsp; If you are using an alternative representation that keeps traceback information in a separate matrix you can return both matrices.  You will need to modify the unit tests.  Make sure your documentation includes a description of your representation and how you tested it.

##### Documentation and Code

Needleman-Wunsch algorithm. Generate two numpy arrays, one for the score and one for the traceback. Fill out each matrix simultaneously. For filling in core cells, loop through rows and columns and choose the max score from the upper cell, left cell, and diagonal cell. Tag according to the max chosen.

In [159]:
def dp_matrix(S, T, score, gp):
    # YOUR CODE HERE
    seq1 = str(S)
    seq2 = str(T)
    
    rows = len(seq1) + 1
    cols = len(seq2) + 1
    
    D = np.zeros((rows, cols), dtype = np.int32)
    T = np.zeros((rows, cols), dtype = np.int32)
    
    D[0][0] = 0
    
    for i in range(1, rows):
        D[i][0] = D[i-1][0] - gp
        T[i][0] = UP

    for j in range(1, cols):
        D[0][j] = D[0][j-1] - gp
        T[0][j] = LEFT
        
    for i in range(1, rows):
        for j in range(1, cols):
            above = mark(D[i-1][j] - gp, UP)
            left = mark(D[i][j-1] - gp, LEFT)
            diag = mark(D[i-1][j-1] + score[seq1[i-1], seq2[j-1]], DIAG)
            directions = [above, left, diag]
            max_val = max(value(above), 
                          value(left), 
                          value(diag))
            D[i][j] = max_val
            
            tags = [tag(i) for i in directions if value(i) == max_val][0]
            T[i][j] = tags
            
    return D, T

In [124]:
mark(-80, LEFT)

-318

In [312]:
D, T = dp_matrix(seq1, seq2, SubstitutionMatrix('BLOSUM50'), 8)

In [161]:
D

array([[  0,  -8, -16, -24, -32, -40, -48, -56, -64, -72, -80],
       [ -8,  -2,  -9, -17, -25, -33, -41, -49, -57, -65, -73],
       [-16, -10,  -3,  -4, -12, -20, -28, -36, -44, -52, -60],
       [-24, -18, -11,  -6,  -7, -15,  -5, -13, -21, -29, -37],
       [-32, -14, -18, -13,  -8,  -9, -13,  -7,  -3, -11, -19],
       [-40, -22,  -8, -16, -16,  -9, -12, -15,  -7,   3,  -5],
       [-48, -30, -16,  -3, -11, -11, -12, -12, -15,  -5,   2],
       [-56, -38, -24, -11,  -6, -12, -14, -15, -12,  -9,   1]], dtype=int32)

In [165]:
T

array([[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2],
       [1, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2],
       [1, 1, 1, 3, 3, 2, 3, 2, 2, 2, 2],
       [1, 3, 3, 3, 3, 3, 1, 3, 3, 2, 2],
       [1, 1, 3, 2, 1, 3, 3, 1, 3, 3, 2],
       [1, 1, 1, 3, 2, 3, 3, 3, 1, 1, 3],
       [1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3]], dtype=int32)

In [327]:
print_matrix(D, T)

• 0 ↤ -8 ↤ -16 ↤ -24 ↤ -32 ↤ -40 ↤ -48 ↤ -56 ↤ -64 ↤ -72 ↤ -80 

↥ -8 ⇖ -2 ⇖ -9 ↤ -17 ↤ -25 ↤ -33 ↤ -41 ↤ -49 ↤ -57 ↤ -65 ↤ -73 

↥ -16 ↥ -10 ⇖ -3 ⇖ -4 ↤ -12 ↤ -20 ↤ -28 ↤ -36 ↤ -44 ↤ -52 ↤ -60 

↥ -24 ↥ -18 ↥ -11 ⇖ -6 ⇖ -7 ↤ -15 ⇖ -5 ↤ -13 ↤ -21 ↤ -29 ↤ -37 

↥ -32 ⇖ -14 ⇖ -18 ⇖ -13 ⇖ -8 ⇖ -9 ↥ -13 ⇖ -7 ⇖ -3 ↤ -11 ↤ -19 

↥ -40 ↥ -22 ⇖ -8 ↤ -16 ↥ -16 ⇖ -9 ⇖ -12 ↥ -15 ⇖ -7 ⇖ 3 ↤ -5 

↥ -48 ↥ -30 ↥ -16 ⇖ -3 ↤ -11 ⇖ -11 ⇖ -12 ⇖ -12 ↥ -15 ↥ -5 ⇖ 2 

↥ -56 ↥ -38 ↥ -24 ↥ -11 ⇖ -6 ⇖ -12 ⇖ -14 ⇖ -15 ⇖ -12 ⇖ -9 ⇖ 1 



##### Autograder Tests 

For these tests, the first cell creates the two short strings used in the example in the lecture notes, and the second calls the `dp_matrix` function.  You can uncomment the third cell and execute it to make sure the matrix looks like the one in the notes.  The autograder will test to see if the matrix has the expected shape and that the last cell has the expected alignment score.

In [338]:
from skbio import Protein

seq1 = Protein('PAWHEAE')
seq2 = Protein('HEAGAWGHEE')

In [387]:
D, T = dp_matrix(seq1, seq2, SubstitutionMatrix('BLOSUM50'), 8)

In [282]:
D = dp_matrix(seq1, seq2, SubstitutionMatrix('BLOSUM50'), 8)

In [388]:
print_matrix(D, T, str(seq1), str(seq2))

• 0 ↤ -8 ↤ -16 ↤ -24 ↤ -32 ↤ -40 ↤ -48 ↤ -56 ↤ -64 ↤ -72 ↤ -80 

↥ -8 ⇖ -2 ⇖ -9 ↤ -17 ↤ -25 ↤ -33 ↤ -41 ↤ -49 ↤ -57 ↤ -65 ↤ -73 

↥ -16 ↥ -10 ⇖ -3 ⇖ -4 ↤ -12 ↤ -20 ↤ -28 ↤ -36 ↤ -44 ↤ -52 ↤ -60 

↥ -24 ↥ -18 ↥ -11 ⇖ -6 ⇖ -7 ↤ -15 ⇖ -5 ↤ -13 ↤ -21 ↤ -29 ↤ -37 

↥ -32 ⇖ -14 ⇖ -18 ⇖ -13 ⇖ -8 ⇖ -9 ↥ -13 ⇖ -7 ⇖ -3 ↤ -11 ↤ -19 

↥ -40 ↥ -22 ⇖ -8 ↤ -16 ↥ -16 ⇖ -9 ⇖ -12 ↥ -15 ⇖ -7 ⇖ 3 ↤ -5 

↥ -48 ↥ -30 ↥ -16 ⇖ -3 ↤ -11 ⇖ -11 ⇖ -12 ⇖ -12 ↥ -15 ↥ -5 ⇖ 2 

↥ -56 ↥ -38 ↥ -24 ↥ -11 ⇖ -6 ⇖ -12 ⇖ -14 ⇖ -15 ⇖ -12 ⇖ -9 ⇖ 1 



In [283]:
assert D.shape == (len(seq1)+1, len(seq2)+1)

In [166]:
assert D[len(seq1), len(seq2)] == 1

In [ ]:
# since separate matrix from traceback

assert D[len(seq1), len(seq2)] == 1

### Part 5: &nbsp; Traceback 

Fill in the body of the function below so it generates a set of alignment strings using the traceback information from the dynamic programming algorithm.  The arguments to the function are the matrix created by a call to `dp_matrix` and the two sequences.

The function should return two strings, the aligned version of the first sequence and the aligned version of the second sequence.  Assuming D is the matrix created by aligning the sequences from the lecture notes:
```
>>> traceback(D, seq1, seq2)
('HEAGAWGHE-E', '--P-AW-HEAE')
```

An optional third argument is the substitution matrix used to create the alignment.  If this argument is given the function should look up the similarity of pairs of aligned letters and return a third string based on these similarities:
```
>>> traceback(D, seq1, seq2, SubstitutionMatrix('BLOSUM50'))
('HEAGAWGHE-E', '    :: :: :', '--P-AW-HEAE')
```

(note that the similarity character is a space if one of the alignments has a gap character)


##### Documentation and Code

Based on algorithm from IAB book. True while row or column are > 0, starting from bottom right corner of traceback matrix and following path towards D[0][0]. Function goes through matrix and if current cell is diagonal, take a letter from each sequence to alignment string and go up diagonally one cell. If current == left, add letter from sequence 2 and gap to sequence 1 and vice versa for if current == up.

if scores is given, add a space if i or j is a gap '-' and get output from similarity function in SimilarityMatrix for two given sequences.

In [419]:
def traceback(matrix, S, T, scores=None):
    # YOUR CODE HERE
    vert = str(T)
    horiz = str(S)
    
    S_align = ''
    T_align = ''
    
    row = matrix.shape[0]-1
    col = matrix.shape[1]-1
    
    while row > 0 or col > 0:
        
        curr = matrix[row][col]
        
        if curr == DIAG:
            S_align += horiz[row-1]
            T_align += vert[col-1]
            row -= 1
            col -= 1
        elif curr == LEFT:
            T_align += vert[col-1]
            S_align += '-'
            col -= 1
        elif curr == UP:
            S_align += horiz[row-1]
            T_align += '-'
            row -=1
        
    seq1 = ''.join(list(S_align)[::-1])
    seq2 = ''.join(list(T_align)[::-1])
    
    alignment = ''
    
    if scores != None:
        for i, j in zip(seq1, seq2):
            if i == '-' or j == '-':
                alignment += ' '
            else:
                alignment += scores.similarity(i, j)
            
        return seq1, alignment, seq2
    else:
        return seq1, seq2

In [420]:
traceback(T, seq1, seq2)

('-PA--W-HEAE', 'HEAGAWGHE-E')

##### Autograder Tests 

seq2 seems to come out right, seq1 comes out partially right....

These tests use the same sequences defined for the `dp_matrix` test.

In [342]:
D = dp_matrix(seq1, seq2, SubstitutionMatrix('BLOSUM50'), 8)

In [ ]:
assert traceback(D, seq1, seq2, SubstitutionMatrix('BLOSUM50')) == ('HEAGAWGHE-E', '    :: :: :', '--P-AW-HEAE')

In [ ]:
assert traceback(D, seq1, seq2) == ('HEAGAWGHE-E', '--P-AW-HEAE')

### Experiments (Optional)

The function below puts all the pieces together.  Call `align` with two sequences, a substitution matrix, and a gap penalty, and it will use the dynamic programming algorithm to align the sequences and print the results.  The return value will be the alignment score.

In [ ]:
def align(seq1, seq2, scores, gp):
    D = dp_matrix(seq1, seq2, scores, gp)
    t,m,b = traceback(D, seq1, seq2, scores)
    print(t)
    print(m)
    print(b)
    return value(D[len(seq1),len(seq2)])

If you want to test your functions on some longer sequences download the FASTA files from Canvas and save them in your project directory.

Try doing the alignments with different substitution matrices and gap penalties to see if they have much of an effect.

##### Hemoglobin 

Hemoglobin transports oxygen in the blood stream and is highly conserved across all animals.  This small data set has one of the proteins used to form hemoglobin from four different animals (human, zebrafish, mouse, and chicken).  The alignment isn't very interesting, but the similarity string shows how similar the proteins are.

In [ ]:
hemoglobin = list(skbio.read('hemoglobin.fasta', format='fasta'))

In [ ]:
for seq in hemoglobin:
    print(seq.metadata['description'], 'length:', len(seq))

In [ ]:
align(hemoglobin[0], hemoglobin[1], SubstitutionMatrix('GONNET'), 8)

##### p53 

P53 is a tumor supressor, also highly conserved.  This data set has proteins from human, mouse, and chimpanzee.  It's included as a "stress test" -- how well does your function work on two sequences that are each about 400 letters long?

In [ ]:
p53 = list(skbio.read('p53.fasta', format='fasta'))

In [ ]:
for seq in p53:
    print(seq.metadata['description'], 'length:', len(seq))

In [ ]:
align(p53[0], p53[1], SubstitutionMatrix('GONNET'), 8)

##### Histones 

This data set is from a site that has a database of groups of conserved proteins.  I downloaded a data set that has histone genes.  The data set doesn't include species information, but you can find that by looking up the IDs at ensembl.org (the prefix "ENS" in the sequence names signifies these IDs are from Ensembl, the European equivalent of NCBI).

These sequences vary in length more than the previous data sets, so it will be interesting to see where the algorithm places the gaps.

In [ ]:
h2 = list(skbio.read('histones.fasta', format='fasta'))

In [ ]:
align(h2[0], h2[5], SubstitutionMatrix('GONNET'), 8)

### Extensions 

Here are some ideas for some ways to extend the project if you're interested.

Implement local alignment:  change the initial conditions on the left column and top row, change your function record the location of the maxium score, and have the traceback report the highest scoring local alignment.

Implement affine gap penalties, where there are separate costs for opening a gap and extending a gap.